# QF620 Part III

Prathmesh Desai  
Rahul Sreeram  
Srivatsa Mitragotri  
Nandini Agarwal  
Harshita Sachdev  

# Import Library

In [4]:
import pandas as pd
import numpy as np

import datetime as dt

import matplotlib.pylab as plt

from scipy import interpolate
from scipy.stats import norm
from scipy.optimize import brentq
from scipy.integrate import quad

# Import Data

In [6]:
# Discount Rate
rate_df = pd.read_csv('zero_rates_20201201.csv')

# SPX General Data
spx_df = pd.read_csv('SPX_options.csv')
spx_df['strike_price'] = spx_df['strike_price']/1000
spx_df['mid_price'] = (spx_df['best_bid'] + spx_df['best_offer'])/2

# SPX Maturity Data
spx = spx_df[(spx_df.exdate == 20210115)]

# Time To Maturity
today = dt.date(2020, 12, 1)
exdate2 = dt.date(2021, 1, 15)
T = (exdate2-today).days/365.0

# Discount Rate Interpolation
x = rate_df['days']
y = rate_df['rate']
f = interpolate.interp1d(x,y)
r = f(T*365)/100

S = 3662.45
K = 3660

# Models

In [8]:
# Black-Scholes Model
def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BlackScholesPut(S, K, r, sigma, T):
    return BlackScholesCall(S,K,r,sigma,T)- S + K*np.exp(-r*T)

# Bachelier Model
def BachelierCall(S, K, r, sigma, T):
    d = (S-K) / (S*sigma*np.sqrt(T))
    disc = np.exp(-r*T)
    return disc*((S-K)*norm.cdf(d)+S*sigma*np.sqrt(T)*norm.pdf(d))

def BachelierPut(S, K, r, sigma, T):
    d = (S-K) / (S*sigma*np.sqrt(T))
    disc = np.exp(-r*T)
    return disc*((K-S)*norm.cdf(-d)+S*sigma*np.sqrt(T)*norm.pdf(-d))

# SABR Model
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom
    return sabrsigma

# Implied Volatility

In [10]:
# Implied European Options Volatility Model
def impliedCallVolatility(S, K, r, price, T):
    try:
        impliedVol = brentq(lambda x: price -
                        BlackScholesCall(S, K, r, x, T),
                        1e-6, 1)
    except Exception:
        impliedVol = np.nan
 
    return impliedVol

def impliedPutVolatility(S, K, r, price, T):
    try:
        impliedVol = brentq(lambda x: price -
                        BlackScholesPut(S, K, r, x, T),
                        1e-6, 1)
    except Exception:
        impliedVol = np.nan

    return impliedVol

## At The Money Volatility (ExDate: 2021/1/15)
atm_call = spx[(spx.strike_price == K)]
atm_call = atm_call[(atm_call.cp_flag == "C")]
atm_put = spx[(spx.strike_price == K)]
atm_put = atm_put[(atm_put.cp_flag == "P")]
sigma_call = impliedCallVolatility(S, K, r , atm_call['mid_price'].iloc[0], T)
sigma_put = impliedPutVolatility(S, K, r, atm_put['mid_price'].iloc[0], T)
sigma = (sigma_call + sigma_put)/2

# Black Scholes Model Integrated Variance

In [12]:
# Black Scholes Model Integrated Variance Function
def callintegrand(K, S, r, T, sigma):
    price = BlackScholesCall(S, K, r, sigma, T) / K**2
    return price

def putintegrand(K, S, r, T, sigma):
    price = BlackScholesPut(S, K, r, sigma, T) / K**2
    return price

In [13]:
F = S * np.exp(r*T)
I_put = quad(lambda x: putintegrand(x, S, r, T, sigma), 0.0, F)
I_call = quad(lambda x: callintegrand(x, S, r, T, sigma), F, 5000)
E_var_BSM = 2*np.exp(r*T)*(I_put[0] + I_call[0])

print('Black Scholes Model Integrated Variance: %.9f' % E_var_BSM)

Black Scholes Model Integrated Variance: 0.004236501


# Black Scholes Model Derivative Pricing

In [15]:
# Black Scholes Model Derivative Pricing Model
sigma_BSM = np.sqrt(E_var_BSM/T)

def payoff_bsm(S,r,T,sigma):
    payoff = S ** (1/3) * np.exp((1/3) * r * T - (1/9) *sigma ** 2 * T) + 1.5 * (np.log(S) + r * T - 0.5 * (sigma ** 2) * T) + 10
    return payoff

payoff_bs = payoff_bsm(S,r,T,sigma_BSM)

print('Black-Scholes Model Derivating Pricing: %.9f' % payoff_bs)

Black-Scholes Model Derivating Pricing: 37.714381258


# Bachelier Model Integrated Variance

In [17]:
# Bachelier Model Integrated Variance Function
def callintegrandb(K, S, r, T, sigma):
    price = BachelierCall(S, K, r, sigma, T) / K**2
    return price

def putintegrandb(K, S, r, T, sigma):
    price = BachelierPut(S, K, r, sigma, T) / K**2
    return price

In [18]:
F = S * np.exp(r*T)
I_put = quad(lambda x: putintegrandb(x, S, r, T, sigma), 0.0, F)
I_call = quad(lambda x: callintegrandb(x, S, r, T, sigma), F, 5000)
E_var_B = 2*np.exp(r*T)*(I_put[0] + I_call[0])

print('Bachelier Model Integrated Variance: %.9f' % E_var_B)

Bachelier Model Integrated Variance: 0.004263876


# Bachelier Model Derivative Pricing

In [20]:
# Bachelier Model Derivative Pricing Model
sigma_B = np.sqrt(E_var_B/T)

x_bachelier = lambda x: (((S + sigma_B*np.sqrt(T)*x)**(1/3))+ (1.5* np.log(S + sigma_B*np.sqrt(T)*x)) +10 )*np.exp(-(x**2/2))
x_b,err = quad(x_bachelier, -(np.inf), np.inf)

derivative_bachelier = np.exp(-r*T)/np.sqrt(2*np.pi)*x_b

print('Bachelier Model Derivative Pricing: %.9f' % derivative_bachelier)

Bachelier Model Derivative Pricing: 37.713596817


# SABR Model Integrated Variance

In [22]:
# SABR Implied Volatility Model
def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, K, r, sabr_vol, T)

def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, K, r, sabr_vol, T)

# SABR Model Integrated Variance Function
def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price

def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price

In [23]:
alpha = 1.817
beta = 0.7
rho = -0.404
nu = 2.790

F = S * np.exp(r*T)
I_put = quad(lambda x: sabrputintegrand(x, S, r, T, alpha, beta, rho, nu), 1e-6, F)
I_call = quad(lambda x: sabrcallintegrand(x, S, r, T, alpha, beta, rho, nu), F, 5000)
E_var_SABR = 2*np.exp(r*T)*(I_put[0] + I_call[0])

print('SABR Model Integrated Variance: %.9f' % E_var_SABR)

SABR Model Integrated Variance: 0.006337324


# SABR Model Derivative Pricing

In [25]:
# SABR Model Derivative Pricing Model
sigma_SABR = np.sqrt(E_var_SABR/T)

x_SABR_bachelier = lambda x: (((S + sigma_SABR*np.sqrt(T)*x)**(1/3))+ (1.5* np.log(S + sigma_SABR*np.sqrt(T)*x)) +10 )*np.exp(-(x**2/2))
x_SABR,err = quad(x_SABR_bachelier, -(np.inf), np.inf)

derivative_SABR_bachelier = np.exp(-r*T)/np.sqrt(2*np.pi)*x_SABR

print('Bachelier Model Derivative Pricing (SABR): %.9f' % derivative_SABR_bachelier)

payoff_SABR_bs = payoff_bsm(S,r,T,sigma_SABR)

print('Black-Scholes Model Derivative Pricing (SABR): %.9f' % payoff_SABR_bs)

Bachelier Model Derivative Pricing (SABR): 37.713596817
Black-Scholes Model Derivative Pricing (SABR): 37.709209373
